In [22]:
import pandas as pd
from recipe_scrapers import scrape_me
import parse_ingredient
import re
import numpy as np
from functools import reduce

In [8]:
urls = ['https://www.bbcgoodfood.com/recipes/halloumi-burgers', 'https://www.allrecipes.com/recipe/158968/spinach-and-feta-turkey-burgers/']

In [5]:
def compile_recipe_info(urls):
    dfs = []
    for url in urls:
        scraper = scrape_me(url)
        ingredient_info = parse_ingredients(scraper.ingredients())

        #Add other info
        ingredient_info["recipe"] = scraper.title()
        ingredient_info["cook_time"] = scraper.total_time()
        ingredient_info["link"] = url

        dfs.append(ingredient_info)

    return pd.concat(dfs)

In [10]:
def parse_ingredients(ingredients):
    data = []
    for ingredient in ingredients:
        print(ingredient)
        info = (ingredient, np.NaN, np.NaN, np.NaN)
        try:
            info = parse_ingredient.parse(ingredient)
            info = info.as_dict()
        except Exception as e:
            pass
            data.append(info)
            continue
        
        #puts original ingredient if low confidence
        if info["confidence"] <= .05:
            data.append((ingredient, np.NaN, np.NaN, np.NaN))
            continue
        
        #puts all information together
        data.append((info["product"], info["quantity"], info["unit"], info["usda_info"]["category"]))
        
    return pd.DataFrame(data, columns=["product", "quantity", "unit", "category"])


In [30]:
def merge_shopping_list(list_):
    quantity = list_.groupby("product").quantity.sum()
    units = list_.groupby("product").unit.unique()
    categories = list_.groupby("product").category.first()

    df_merged = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), [quantity, units, categories])
    return df_merged.sort_values(["category", "product"])

In [11]:
list_ = compile_recipe_info(urls)

250g block halloumi cheese, cut into 8 thick slices
None
1 tbsp olive oil
None
4 brioche buns
None
4 tbsp hummus
None
1 large ripe tomato, thinly sliced
None
4 Butterhead lettuce leaves
None
4 tbsp fresh tomato salsa (most supermarkets stock a version)
None
2 eggs, beaten
None
2 cloves garlic, minced
None
4 ounces feta cheese
None
1 (10 ounce) box frozen chopped spinach, thawed and squeezed dry
None
2 pounds ground turkey
None


In [12]:
list_

,product,quantity,unit,category,recipe,cook_time,link
0,"250g block halloumi cheese, cut into 8 thick s...",NaN,NaN,NaN,Halloumi burgers,20,https://www.bbcgoodfood.com/recipes/halloumi-b...
1,olive oil,1.0,tablespoon,Fats and Oils,Halloumi burgers,20,https://www.bbcgoodfood.com/recipes/halloumi-b...
2,4 brioche buns,NaN,NaN,NaN,Halloumi burgers,20,https://www.bbcgoodfood.com/recipes/halloumi-b...
3,hummus,4.0,tablespoon,None,Halloumi burgers,20,https://www.bbcgoodfood.com/recipes/halloumi-b...
4,tomato,1.0,None,Vegetables and Vegetable Products,Halloumi burgers,20,https://www.bbcgoodfood.com/recipes/halloumi-b...
5,4 Butterhead lettuce leaves,NaN,NaN,NaN,Halloumi burgers,20,https://www.bbcgoodfood.com/recipes/halloumi-b...
6,4 tbsp fresh tomato salsa (most supermarkets s...,NaN,NaN,NaN,Halloumi burgers,20,https://www.bbcgoodfood.com/recipes/halloumi-b...
0,eggs,2.0,None,Dairy and Egg Products,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...
1,garlic,2.0,clove,None,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...
2,feta cheese,4.0,ounce,Dairy and Egg Products,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...


In [31]:
final_shopping_list = merge_shopping_list(list_)

In [32]:
final_shopping_list

,quantity,unit,category
product,,,
eggs,2.0,[None],Dairy and Egg Products
feta cheese,4.0,[ounce],Dairy and Egg Products
olive oil,1.0,[tablespoon],Fats and Oils
turkey,2.0,[pound],Poultry Products
tomato,1.0,[None],Vegetables and Vegetable Products
"1 (10 ounce) box frozen chopped spinach, thawed and squeezed dry",0.0,[nan],NaN
"250g block halloumi cheese, cut into 8 thick slices",0.0,[nan],NaN
4 Butterhead lettuce leaves,0.0,[nan],NaN
4 brioche buns,0.0,[nan],NaN
